In [12]:
#  Import required libraries for this Notebook

import numpy as np
import pandas as pd
import os as  sys
from sklearn import  metrics, preprocessing 
from sklearn.metrics import roc_curve, auc , precision_recall_curve , precision_recall_fscore_support, precision_score
from sklearn.model_selection import KFold
from nimfa.utils.linalg import *
from collections import defaultdict
from os.path import dirname
from os.path import join
from skfusion import fusion as skf
from skfusion.fusion import ObjectType, Relation, FusionGraph

In [ ]:
def residuals(V, V_hat):
        """
        Return residuals matrix between the target matrix and its NMF estimate.
        """
        
        return(V - V_hat)

def rss(V,V_hat):
        """
        Compute Residual Sum of Squares (RSS) between NMF estimate and
        target matrix [Hutchins2008]_.
        """        
        X = residuals(V, V_hat)
        return multiply(X, X).sum()

def evar(target, _target):
        """
        Compute the explained variance of the NMF estimate of the target matrix.
        """       
        V = target
        V_hat=_target
        return 1. - rss(V,V_hat) / multiply(V, V).sum()
def DiffusionKernel(AdjMatrix):
    # 1.Computes Degree matrix  - diagonal matrix with diagonal entries = raw sums of adjacency matrix 
    DegreeMatrix = np.diag(np.sum(AdjMatrix, axis=1))
    # 2. Computes negative Laplacian H = AdjMatrix - DegreeMatrix
#     H = np.subtract(AdjMatrix, DegreeMatrix)
    H = np.subtract(AdjMatrix,DegreeMatrix)
    # 3. Computes matrix exponential: exp(beta*H)
    K = sp.linalg.expm(0.2*H)
    return K
def load_ppi_data (GRNData,ppi_file):
    """
    Read protein protein interaction data and convert it into Adjacency matrix
    
    return : Adjacency of PPI
    """
    ppi=np.loadtxt(ppi_file, dtype=np.int32)
    PPI=np.zeros(((GRNData['nvertices']),(GRNData['nvertices'])))    
    for i in range(0,ppi.shape[0]):
         PPI[ppi[i][0],ppi[i][1]]=1
    return PPI
def load_tf_tfi_data (GRNData,tf_tfi_file):
    """
    Read  interaction data between known transcription factors in this data 
    
    return : Adjacency of TF-TFI
    """
    tf_tfi=np.loadtxt(tf_tfi_file, dtype=np.int32)
    TF_TFI=np.zeros(((GRNData['nsources']),(GRNData['nsources'])))    
    for i in range (0,tf_tfi.shape[0]):
         TF_TFI[tf_tfi[i,0], tf_tfi[i,1]]=1
            
    return TF_TFI
def load_tf_gene_interaction_data(GRNData):
    
    """
     Generate an adjacency matrix from the known regulations of E.coli benchmark data
    
    return : Adjacency of TF-Gene interactions
    """
    
    i,j,sourceIndices = np.unique(GRNData['edges'][:,0], return_index=True, return_inverse=True)
    row_count,col_count=[GRNData['nvertices'], GRNData['nsources']]
    Gold=np.zeros([row_count,col_count])
    edges=GRNData['edges']
    for i in range(0,GRNData['nedges']):
        Gold[edges[i,1],sourceIndices[i]] = 1 
        
    return Gold

def load_ecoli_data ():
   
    """
    Read   all E.coli genomic data and build a data structure GRNData for further processing.
    
    return :  Dictionary of Ecoli datasets and other attributes
    
    """
    DATA_DIR="./data/"
    expr_file=DATA_DIR + 'expression.txt'
    expr_file='/home/csserver/sirene-1.1/data/ecoli/ecoli_expression_data'
    reg_file=DATA_DIR + 'regulation.txt'    
    go_sem_file =DATA_DIR + 'EcK12SemSim_4345.txt'
    pp_interaction_file=DATA_DIR +'ppi_interaction_strings.txt'
    tf_tfi_file=DATA_DIR +'tf_tf_interaction.txt'
    GRNData=buildRegulationData(expr_file,reg_file,pp_interaction_file,go_sem_file,tf_tfi_file)
    
    return GRNData
def get_genes():
    """
    Read   Complete list of genes for E. coli benchmark.
    
    return :  List of genes.
    
    """
    DATA_DIR="./data/"
    genes=np.genfromtxt(DATA_DIR + "gene_names.txt",dtype='object')
    
    
    return genes
def matrix_cmpletion (fuser):
    
    """
    fuser object containing relations and their latent factors.
    
    return :  Reconstructed   (R12_hat) from  low rank matrix  facors
    
    """
    S12=fuser.backbones_[gene_tf_relation][i]
    G1=fuser.factors_[gene][i]
    G2=fuser.factors_[tf][i]
    target_hat=np.dot(G1, np.dot(S12, G2.T))  
    
    return 

In [15]:
def  buildRegulationData (expr_file,regulation_file,ppi_file,go_sem_file,tf_tfi_file):
    
    """
        This function reads required data files to build a datastructure containing all neceaasy information for GRN inference
    
        return :  Reconstructed   (R12_hat) from  low rank matrix  facors
    
    """
        
    df_exp = pd.read_csv(expr_file,sep='\t', header=None,index_col=None)
    exp_matrix = df_exp.as_matrix()
    df_regulation=pd.DataFrame.from_csv(regulation_file,sep='\t', header=None,index_col=None)
    df_regulation_matrix=df_regulation.as_matrix()
    go_sem=pd.read_csv(go_sem_file, sep='\t',header=None,index_col=None)
    go_sem=go_sem.as_matrix()
    tf_tfi=np.loadtxt(tf_tfi_file, dtype=np.int32)   
    edges=df_regulation_matrix    
    edges=edges-1
    knownTargets= np.unique(edges[:,1])
    nrows, ncols = exp_matrix.shape    
    toPredict = np.setdiff1d(np.arange(nrows),knownTargets)
    sources = np.unique(edges[:,0])
    nsource = sources.size
    nvertices , nfeatures= exp_matrix.shape
    GRNData = { 'features'  :    exp_matrix , 
                'edges'     :    edges,
                 'go_sem'   :    go_sem,                 
                'nvertices' :    nvertices,
                'nfeatures' :     nfeatures,                                 
                'nedges' :       edges.shape[0],
                'knownTargets' : knownTargets,   
                'toPredict' :    toPredict,                 
                'sources'   :    sources,
                'nsources' :     nsource                    
          }
    GRNData['Gold']=load_tf_gene_interaction_data(GRNData)
    """
        Compution of  diffusion kernel commneted out becuase of huge CPU time it takes
    
    """
#     GRNData['ppi']=DiffusionKernel(load_ppi_data(GRNData, ppi_file))
#     GRNData['tf_tfi']=DiffusionKernel(load_tf_tfi_data(GRNData,tf_tfi_file))
    GRNData['ppi']=load_ppi_data(GRNData, ppi_file)
    GRNData['tf_tfi']=load_tf_tfi_data(GRNData,tf_tfi_file)

    return  GRNData

In [16]:
def main (GRNData):
    gene=skf.ObjectType("Gene",250)
    go=skf.ObjectType("GO",200)
    exp_cond=skf.ObjectType("Experimental condition",15)
    tf=skf.ObjectType("TF",15)
    n_folds = 5
    n_genes = len(get_genes())
    print(n_genes)
    cv = KFold(n_folds=n_folds)
    fold_mse = np.zeros(n_folds)
    precision_avg=np.zeros(n_folds)
    recall_avg=np.zeros(n_folds)
    geneTF_mask = np.zeros_like(GRNData['Gold']).astype('bool')
    print(geneTF_mask.shape)
    knownTargets=GRNData['knownTargets']
    
    relations = [
            skf.Relation(GRNData['Gold'],gene,tf),           
            skf.Relation(GRNData['go_sem'],gene,go),
            skf.Relation(GRNData['features'],gene,exp_cond),
             skf.Relation(GRNData['ppi'],gene,gene),
            skf.Relation(GRNData['tf_tfi'],tf,tf)                        
    ]
    fusion_graph = skf.FusionGraph(relations)
    fuser = skf.Dfmc(max_iter=100, n_run=1, init_type="random_vcol", random_state=0)
    gold=0
    for i, (train_idx, test_idx) in cv.split(n_genes):
        geneTF_mask[:] = False
        geneTF_mask[test_idx, :] = True
        fusion_graph[gene][tf][0].mask = geneTF_mask
        fuser.fuse(fusion_graph)
        pred_ann = fuser.complete(fuser.fusion_graph[gene][tf][0])[test_idx]        
        true_ann = GRNData['Gold'][test_idx]        
        threshold=0.122415
        print (np.sum(true_ann>0), np.sum(pred_ann>threshold))
        C = np.where(pred_ann >= threshold, 1, 0)      
        print(metrics.confusion_matrix(true_ann.flatten(),C.flatten()))
        print("F1 score")
        print (metrics.f1_score(true_ann.flatten(),C.flatten()))
        print("Precision score")
        print (metrics.precision_score(true_ann.flatten(),C.flatten()))
        print("Recall Score")
        print (metrics.recall_score(true_ann.flatten(),C.flatten()))
        fold_mse[i] = metrics.mean_squared_error(pred_ann, true_ann)
        precision_avg[i]=metrics.precision_score(true_ann.flatten(),C.flatten())
        recall_avg[i]=metrics.recall_score(true_ann.flatten(),C.flatten())
        gold=gold+np.sum(C>0)
    pred_ann = fuser.complete(fuser.fusion_graph[gene][tf][0])
    C = np.where(pred_ann[knownTargets] >= threshold, 1, 0)
    print("SCore for Known Targets")
    print (np.sum(pred_ann>=threshold))
    print (metrics.confusion_matrix(GRNData['Gold'][knownTargets].flatten(),C.flatten()))
    print("Average Precision %5.4f" % np.mean(precision_avg))
    print ("Average Recall %5.4f" % np.mean( recall_avg))  
    print("MSE: %5.4f" % np.mean(fold_mse))     
    print ("Total True interactions recovered %3.2f" % gold)    
    pred_ann = fuser.complete(fuser.fusion_graph[gene][tf][0])
    return pred_ann,true_ann

In [17]:
if __name__ == '__main__':
  
       
    ecoli_data=load_ecoli_data()      
    predicted,gold=main(ecoli_data)    

    

4345


TypeError: __init__() got an unexpected keyword argument 'n_folds'